In [7]:
#%cd ..
%load_ext autoreload
%autoreload 2

from pykitti.tracking import KittiTrackingLabels
from datasets.PyKitti2Dataset import PyKitti2
from hugeica import *
from Mine import *
from hugeica import Adam_Lie
from Crop import crop

import imageio
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import meshplot as mp

from sklearn.neighbors import NearestNeighbors
from AutoCalibration import *

basedir = "/home/ios/data3/kitti/tracking/training"
seq = "0000"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [8]:
kitti = PyKitti2(basedir, seq, with_labels="box")   

files 154


In [9]:
datas = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(10)]

kitti1 = datas[0]
initRT = kitti1.calib.T_cam2_velo.copy()

files 154
files 447
files 233
files 144
files 314
files 297


/home/ios/anaconda3/envs/pytorch/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


files 270
files 800
files 390
files 803


In [10]:
# Kitti dataset is recalibrated on different days, so we can just test on sequences that have the same calibration
datas_test1 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(10,14)]

files 294
files 373
files 78
files 340


In [11]:
datas_test2 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(14,18)]

files 106
files 376
files 209
files 145


In [12]:
datas_test3 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(18,20)]

files 339
files 1059


In [13]:
datas_test4 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(20,21)]

files 837


In [14]:
datas_test = [datas_test1, datas_test2, datas_test3, datas_test4]

In [15]:
calib0 = datas_test1[0].calib.T_cam2_velo
calib1 = datas_test2[0].calib.T_cam2_velo
calib2 = datas_test3[0].calib.T_cam2_velo
calib3 = datas_test4[0].calib.T_cam2_velo

calib = [calib0, calib1, calib2, calib3]

datas_test

[[PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0010)[294],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0011)[373],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0012)[78],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0013)[340]],
 [PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0014)[106],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0015)[376],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0016)[209],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0017)[145]],
 [PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0018)[314],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0019)[1059]],
 [PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0020)[837]]]

# Testing number of frames (no sampling)

In [27]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         patch_size=96, hidden_size=32,  
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    interval_r, interval_t = randomRT(0.6, 2)

    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad in [(3000, 1e-3, False), (1000, 1e-4, True), (1000, 1e-5, True)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.965,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True)

    log.append((r, interval_t, interval_r))
    
metrics = [] 
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.35442164  0.14312516 -0.24041439] r:  [-0.9326909   0.48453533  0.11656838] 

#   0: nll: 0.31/0.43, mi:  2.009/ 1.354, acc: 87.5/80.8, eps: 0.44688, R=[-0.88, 0.54, 0.06], t=0.45, Q_cos=1.03
#  30: nll: 0.54/0.38, mi:  0.828/ 1.554, acc: 72.9/86.0, eps: 0.08267, R=[-0.77, 0.75, 0.01], t=0.36, Q_cos=1.08
Saved checkpoint
#  60: nll: 0.43/0.36, mi:  1.674/ 2.113, acc: 81.2/85.2, eps: 0.06509, R=[-0.69, 0.79, -0.15], t=0.30, Q_cos=1.06
Saved checkpoint
#  90: nll: 0.35/0.36, mi:  2.496/ 1.847, acc: 81.2/84.8, eps: 0.03896, R=[-0.77, 0.84, -0.3], t=0.26, Q_cos=1.17
# 120: nll: 0.19/0.40, mi:  3.823/ 1.726, acc: 95.8/82.3, eps: 0.01016, R=[-0.83, 0.78, -0.9], t=0.25, Q_cos=1.45
# 150: nll: 0.41/0.37, mi:  1.458/ 1.970, acc: 85.4/85.4, eps: 0.00917, R=[-0.74, 0.74, -1.16], t=0.24, Q_cos=1.56
# 180: nll: 0.32/0.35, mi:  2.228/ 1.990, acc: 89.6/85.0, eps: 0.00066, R=[-0.93, 0.72, -1.3], t=0.24, Q_cos=1.75
# 210: nll: 0.46/0.33, mi:  1.450/ 2.247, acc: 75.0/85.4, eps: 0.00

(array([4.1726967 , 0.39494901]), array([1.60984127, 0.21298023]))

In [24]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         patch_size=96, hidden_size=32,  
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    interval_r, interval_t = randomRT(0.6, 2)

    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad in [(3000, 1e-3, False), (1000, 1e-4, True), (1000, 1e-5, True)]:
    
        #ac.load_state_dict(torch.load("checkpoints/checkpoint_best_mi.pth.tar"))
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.8,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.35442164  0.14312516 -0.24041439] r:  [-0.9326909   0.48453533  0.11656838] 

#   0: nll: 0.31/0.43, mi:  2.009/ 1.354, acc: 87.5/80.8, eps: 0.44688, R=[-0.88, 0.54, 0.06], t=0.45, Q_cos=1.03
#  30: nll: 0.48/0.40, mi:  0.665/ 1.573, acc: 87.5/81.3, eps: 0.06513, R=[-0.57, 0.84, -0.55], t=0.38, Q_cos=1.16
Saved checkpoint
#  60: nll: 0.34/0.45, mi:  1.419/ 1.260, acc: 89.6/80.4, eps: 0.01519, R=[-0.56, 0.72, -0.35], t=0.37, Q_cos=0.97
#  90: nll: 0.34/0.39, mi:  1.876/ 1.670, acc: 89.6/84.0, eps: 0.00902, R=[-0.76, 0.74, -0.24], t=0.37, Q_cos=1.08
Saved checkpoint
# 120: nll: 0.48/0.45, mi:  1.225/ 1.222, acc: 77.1/78.5, eps: 0.01767, R=[-0.62, 0.73, -0.09], t=0.36, Q_cos=0.96
# 150: nll: 0.90/0.35, mi: -0.344/ 1.949, acc: 54.2/87.3, eps: 0.01691, R=[-0.85, 0.63, -0.48], t=0.37, Q_cos=1.16
Saved checkpoint
# 180: nll: 0.37/0.41, mi:  1.224/ 1.428, acc: 85.4/81.2, eps: 0.00792, R=[-0.79, 0.75, -0.69], t=0.37, Q_cos=1.29
# 210: nll: 0.36/0.47, mi:  0.841/ 0.933, acc:

(array([1.42538629, 0.21387151]), array([0.12297847, 0.03840241]))

In [26]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         patch_size=96, hidden_size=32,  
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    interval_r, interval_t = randomRT(0.6, 2)

    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad in [(3000, 1e-3, False), (1000, 1e-4, True), (1000, 1e-5, True)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.5,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.35442164  0.14312516 -0.24041439] r:  [-0.9326909   0.48453533  0.11656838] 

#   0: nll: 0.85/0.46, mi:  0.408/ 1.118, acc: 62.5/77.9, eps: 0.43599, R=[-0.88, 0.54, 0.06], t=0.44, Q_cos=1.03
#  30: nll: 0.70/0.46, mi:  0.324/ 1.191, acc: 62.5/81.7, eps: 0.06089, R=[-0.55, 0.79, 0.33], t=0.37, Q_cos=1.02
Saved checkpoint
#  60: nll: 0.46/0.40, mi:  1.157/ 1.458, acc: 75.0/81.9, eps: 0.01431, R=[-0.49, 0.76, 0.41], t=0.36, Q_cos=0.99
Saved checkpoint
#  90: nll: 0.27/0.44, mi:  2.760/ 1.057, acc: 93.8/83.8, eps: 0.01330, R=[-0.57, 0.84, 0.56], t=0.35, Q_cos=1.16
# 120: nll: 0.26/0.46, mi:  3.195/ 0.996, acc: 89.6/81.9, eps: 0.00985, R=[-0.47, 0.82, 0.87], t=0.34, Q_cos=1.29
# 150: nll: 0.38/0.40, mi:  2.138/ 1.334, acc: 85.4/84.0, eps: 0.00658, R=[-0.72, 0.77, 0.97], t=0.34, Q_cos=1.43
# 180: nll: 0.35/0.44, mi:  2.075/ 1.072, acc: 83.3/80.4, eps: 0.00178, R=[-0.94, 0.76, 0.76], t=0.34, Q_cos=1.43
# 210: nll: 0.35/0.43, mi:  1.899/ 1.216, acc: 87.5/82.5, eps: 0.0300

(array([1.79576277, 0.43476072]), array([1.03010875, 0.35383704]))

In [25]:
np.random.seed(3)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         patch_size=96, hidden_size=32,  
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    interval_r, interval_t = randomRT(0.6, 2)
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad in [(3000, 1e-3, False), (1000, 1e-4, True), (1000, 1e-5, True)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [ 0.17897286 -0.26581526  0.21150588] r:  [ 0.36345127 -1.90407247  0.23541635] 

#   0: nll: 0.92/0.77, mi: -0.577/-0.302, acc: 50.0/57.7, eps: 0.39319, R=[0.42, -1.85, 0.18], t=0.39, Q_cos=1.90
#  30: nll: 0.89/1.07, mi: -0.350/-1.152, acc: 52.1/52.1, eps: 0.10426, R=[0.91, -2.4, -0.17], t=0.49, Q_cos=2.57
#  60: nll: 0.78/1.14, mi:  0.173/-1.285, acc: 54.2/51.9, eps: 0.01957, R=[1.05, -2.57, -0.02], t=0.51, Q_cos=2.77
#  90: nll: 0.72/1.01, mi: -0.190/-0.476, acc: 56.2/55.0, eps: 0.01248, R=[1.17, -2.72, -0.2], t=0.52, Q_cos=2.97
# 120: nll: 1.07/1.00, mi: -1.081/-0.780, acc: 50.0/54.8, eps: 0.00247, R=[1.42, -2.53, -0.27], t=0.53, Q_cos=2.91
# 150: nll: 0.89/1.07, mi: -0.307/-0.864, acc: 60.4/51.7, eps: 0.06783, R=[1.24, -2.62, 0.27], t=0.46, Q_cos=2.92
# 180: nll: 1.13/1.16, mi: -0.905/-1.193, acc: 45.8/50.0, eps: 0.00072, R=[1.17, -2.94, 0.73], t=0.46, Q_cos=3.25
# 210: nll: 1.19/1.11, mi: -1.252/-1.332, acc: 39.6/52.1, eps: 0.00993, R=[1.18, -2.82, 0.72], t=0.47

(array([2.6642629, 0.4665285]), array([3.21789635, 0.40831156]))

In [15]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         patch_size=96, hidden_size=32,  
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    interval_r, interval_t = randomRT(0.6, 2)

    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad in [(3000, 1e-3, False), (1000, 1e-4, True), (1000, 1e-5, True)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:2], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

#   0: nll: 0.95/0.72, mi: -0.492/ 0.197, acc: 60.4/61.9, eps: 0.57027, R=[-0.32, -0.38, -0.74], t=0.57, Q_cos=0.89
#  30: nll: 0.54/0.60, mi:  0.349/ 0.495, acc: 75.0/68.1, eps: 0.07004, R=[-0.91, 0.12, -1.66], t=0.50, Q_cos=1.90
Saved checkpoint
#  60: nll: 0.39/0.48, mi:  1.404/ 1.254, acc: 85.4/76.0, eps: 0.07334, R=[-1.45, 0.04, -1.43], t=0.42, Q_cos=2.04
Saved checkpoint
#  90: nll: 0.83/0.43, mi: -0.670/ 1.356, acc: 56.2/79.6, eps: 0.03922, R=[-1.57, 0.02, -0.67], t=0.39, Q_cos=1.71
Saved checkpoint
# 120: nll: 0.40/0.40, mi:  1.690/ 1.696, acc: 81.2/82.3, eps: 0.06201, R=[-1.78, 0.03, 0.31], t=0.32, Q_cos=1.81
Saved checkpoint
# 150: nll: 0.43/0.39, mi:  1.757/ 1.749, acc: 75.0/83.1, eps: 0.00205, R=[-1.59, 0.03, 0.28], t=0.33, Q_cos=1.61
Saved checkpoint
# 180: nll: 0.39/0.40, mi:  0.284/ 1.609, acc: 89.6/82.7, eps: 0.00349, R=[-1.44, 0.07, 0.31], t=0.33, Q_cos=1.47
# 210: nll: 0

(array([0.09167358, 0.03200712]), array([0.01036457, 0.00195376]))

In [22]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         patch_size=96, hidden_size=32,  
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    interval_r, interval_t = randomRT(0.6, 2)

    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad in [(3000, 1e-3, False), (1000, 1e-4, True), (1000, 1e-5, True)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:3], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

#   0: nll: 0.92/0.75, mi: -1.399/-0.046, acc: 41.7/59.2, eps: 0.58986, R=[-0.2, -0.38, -0.74], t=0.59, Q_cos=0.85
#  30: nll: 0.74/0.64, mi:  0.009/ 0.255, acc: 70.8/66.9, eps: 0.11018, R=[-0.59, 0.2, -1.47], t=0.48, Q_cos=1.59
#  60: nll: 0.53/0.47, mi:  1.122/ 1.369, acc: 64.6/77.9, eps: 0.11850, R=[-1.15, -0.05, -1.37], t=0.36, Q_cos=1.78
Saved checkpoint
#  90: nll: 0.63/0.40, mi:  0.508/ 1.643, acc: 70.8/82.5, eps: 0.07976, R=[-1.45, 0.04, -0.53], t=0.28, Q_cos=1.54
Saved checkpoint
# 120: nll: 0.39/0.37, mi:  1.609/ 1.744, acc: 85.4/84.8, eps: 0.00728, R=[-1.34, 0.15, 0.19], t=0.27, Q_cos=1.37
Saved checkpoint
# 150: nll: 0.45/0.40, mi:  1.390/ 1.778, acc: 81.2/80.6, eps: 0.02762, R=[-1.31, 0.08, 0.31], t=0.24, Q_cos=1.35
Saved checkpoint
# 180: nll: 0.52/0.43, mi:  0.860/ 1.180, acc: 72.9/80.0, eps: 0.00078, R=[-1.14, 0.05, 0.46], t=0.24, Q_cos=1.23
# 210: nll: 0.60/0.45, mi:  0.5

(array([0.09307778, 0.03778597]), array([0.01610146, 0.00463618]))

In [17]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         patch_size=96, hidden_size=32,  
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    interval_r, interval_t = randomRT(0.6, 2)

    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad in [(3000, 1e-3, False), (1000, 1e-4, True), (1000, 1e-5, True)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:4], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

#   0: nll: 1.21/0.70, mi: -1.430/ 0.098, acc: 43.8/63.3, eps: 0.57019, R=[-0.32, -0.38, -0.62], t=0.57, Q_cos=0.79
#  30: nll: 0.52/0.49, mi:  0.755/ 1.244, acc: 70.8/76.2, eps: 0.19806, R=[-1.45, 0.07, -0.88], t=0.37, Q_cos=1.70
Saved checkpoint
#  60: nll: 0.42/0.37, mi:  1.349/ 1.713, acc: 83.3/85.0, eps: 0.06120, R=[-1.6, 0.06, 0.04], t=0.31, Q_cos=1.60
Saved checkpoint
#  90: nll: 0.31/0.52, mi:  2.513/ 0.748, acc: 89.6/74.0, eps: 0.04428, R=[-1.7, 0.03, 0.34], t=0.26, Q_cos=1.73
# 120: nll: 0.47/0.41, mi:  1.357/ 1.734, acc: 77.1/81.7, eps: 0.01959, R=[-1.4, 0.03, 0.04], t=0.28, Q_cos=1.40
Saved checkpoint
# 150: nll: 0.41/0.44, mi:  1.547/ 1.244, acc: 77.1/79.0, eps: 0.04460, R=[-1.43, 0.13, -0.01], t=0.24, Q_cos=1.43
# 180: nll: 0.36/0.40, mi:  0.695/ 1.541, acc: 89.6/82.5, eps: 0.03272, R=[-1.3, 0.31, 0.36], t=0.21, Q_cos=1.38
# 210: nll: 0.38/0.50, mi:  2.030/ 1.111, acc: 81.2/

(array([0.14234461, 0.04785769]), array([0.01298192, 0.00394892]))

# Testing number of frames (sampling)

In [10]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, True), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.965,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.57/0.68, mi:  0.494/ 0.199, acc: 67.5/61.5, eps: 0.58518, R=[-0.2, -0.26, -0.74], t=0.58, Q_cos=0.81
#  30: nll: 0.68/0.58, mi: -0.131/ 0.516, acc: 60.0/71.2, eps: 0.01986, R=[-0.36, -0.22, -0.76], t=0.60, Q_cos=0.87
Saved checkpoint
#  60: nll: 0.51/0.56, mi:  0.644/ 0.604, acc: 75.0/73.0, eps: 0.02603, R=[-0.52, -0.14, -0.77], t=0.63, Q_cos=0.94
Saved checkpoint
#  90: nll: 0.61/0.51, mi:  0.527/ 0.825, acc: 60.0/73.7, eps: 0.00534, R=[-0.56, -0.22, -1.05], t=0.64, Q_cos=1.21
Saved checkpoint
# 120: nll: 0.34/0.54, mi:  2.193/ 0.776, acc: 77.5/74.5, eps: 0.00864, R=[-0.75, -0.12, -1.4], t=0.63, Q_cos=1.59
# 150: nll: 0.53/0.52, mi:  0.658/ 0.744, acc: 75.0/73.8, eps: 0.04581, R=[-0.62, -0.15, -1.79], t=0.67, Q_cos=1.90
# 180: nll: 0.42/0.48, mi:  1.563/ 0.873, acc: 82.5/77.7, eps: 0.01055, R=[-0.7, -0.06, -2.35], t=0.68, Q_cos=2.46
Saved check

(array([2.88444716, 0.72867707]), array([1.47743205, 0.26726513]))

In [36]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, True), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.8,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.54/0.59, mi:  0.812/ 0.796, acc: 72.5/67.0, eps: 0.56535, R=[-0.32, -0.26, -0.62], t=0.57, Q_cos=0.75
#  30: nll: 0.43/0.40, mi:  2.249/ 1.554, acc: 70.0/84.8, eps: 0.15381, R=[-1.22, -0.11, -0.71], t=0.41, Q_cos=1.42
Saved checkpoint
#  60: nll: 0.33/0.35, mi:  2.088/ 2.052, acc: 92.5/85.3, eps: 0.14574, R=[-1.99, -0.1, -0.51], t=0.26, Q_cos=2.06
Saved checkpoint
#  90: nll: 0.45/0.46, mi:  1.174/ 1.356, acc: 77.5/79.3, eps: 0.01806, R=[-1.97, -0.05, -0.44], t=0.24, Q_cos=2.02
# 120: nll: 0.31/0.37, mi:  2.150/ 1.674, acc: 90.0/86.0, eps: 0.05670, R=[-1.46, -0.01, 0.09], t=0.30, Q_cos=1.47
# 150: nll: 0.36/0.43, mi:  1.352/ 1.431, acc: 82.5/80.0, eps: 0.03373, R=[-1.56, 0.01, 0.15], t=0.27, Q_cos=1.57
# 180: nll: 0.44/0.40, mi:  1.144/ 1.763, acc: 82.5/83.2, eps: 0.03810, R=[-1.78, 0.06, 0.19], t=0.23, Q_cos=1.79
# 210: nll: 0.73/0.35, mi:  0.1

(array([1.24929896, 0.13886765]), array([0.15227856, 0.03156138]))

In [35]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, True), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.5,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.81/0.65, mi: -0.432/ 0.431, acc: 50.0/63.2, eps: 0.58518, R=[-0.2, -0.26, -0.74], t=0.58, Q_cos=0.81
#  30: nll: 0.35/0.55, mi:  2.139/ 0.656, acc: 82.5/72.0, eps: 0.06740, R=[-0.82, -0.1, -0.55], t=0.52, Q_cos=0.99
Saved checkpoint
#  60: nll: 0.33/0.46, mi:  1.464/ 1.523, acc: 90.0/81.5, eps: 0.09635, R=[-1.35, -0.01, -0.35], t=0.42, Q_cos=1.39
Saved checkpoint
#  90: nll: 0.40/0.40, mi:  1.841/ 1.564, acc: 80.0/84.0, eps: 0.09606, R=[-1.79, 0.04, -0.36], t=0.32, Q_cos=1.82
Saved checkpoint
# 120: nll: 0.55/0.36, mi: -0.331/ 1.810, acc: 85.0/85.2, eps: 0.02820, R=[-1.97, -0.01, -0.35], t=0.29, Q_cos=2.00
Saved checkpoint
# 150: nll: 0.40/0.38, mi:  1.724/ 1.612, acc: 80.0/85.2, eps: 0.00629, R=[-1.94, 0.05, 0.13], t=0.29, Q_cos=1.94
# 180: nll: 0.38/0.42, mi:  1.753/ 1.414, acc: 82.5/81.0, eps: 0.02201, R=[-2.05, 0.04, 0.12], t=0.26, Q_cos=2.0

(array([2.50013183, 0.3860342 ]), array([2.01626725, 0.39491969]))

In [34]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, True), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.81/0.65, mi: -0.432/ 0.431, acc: 50.0/63.2, eps: 0.58518, R=[-0.2, -0.26, -0.74], t=0.58, Q_cos=0.81
#  30: nll: 0.36/0.57, mi:  1.524/ 0.701, acc: 85.0/73.3, eps: 0.03817, R=[-0.38, -0.18, -0.49], t=0.55, Q_cos=0.64
Saved checkpoint
#  60: nll: 0.37/0.53, mi:  1.843/ 0.989, acc: 80.0/79.3, eps: 0.13331, R=[-1.36, -0.09, -0.81], t=0.41, Q_cos=1.59
Saved checkpoint
#  90: nll: 0.27/0.44, mi:  2.282/ 1.342, acc: 92.5/80.5, eps: 0.10632, R=[-2.01, -0.06, -0.77], t=0.30, Q_cos=2.15
Saved checkpoint
# 120: nll: 0.34/0.35, mi:  2.259/ 1.956, acc: 80.0/87.5, eps: 0.00935, R=[-1.7, 0.02, -0.68], t=0.31, Q_cos=1.83
Saved checkpoint
# 150: nll: 0.30/0.43, mi:  2.208/ 1.307, acc: 87.5/81.2, eps: 0.04754, R=[-1.71, 0.01, -0.68], t=0.27, Q_cos=1.84
# 180: nll: 0.38/0.37, mi:  1.762/ 1.824, acc: 82.5/83.2, eps: 0.02313, R=[-1.74, 0.01, -0.7], t=0.24, Q_cos=1.

(array([0.23745036, 0.1217819 ]), array([0.01157156, 0.00939478]))

In [33]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, True), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:2], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.60/0.69, mi:  0.293/ 0.367, acc: 62.5/63.5, eps: 0.56818, R=[-0.32, -0.38, -0.74], t=0.57, Q_cos=0.89
#  30: nll: 0.78/0.61, mi: -0.097/ 0.498, acc: 52.5/69.3, eps: 0.03954, R=[-0.33, 0.07, -1.34], t=0.61, Q_cos=1.38
Saved checkpoint
#  60: nll: 0.76/0.59, mi:  0.023/ 0.294, acc: 60.0/72.2, eps: 0.01720, R=[-0.65, 0.06, -1.3], t=0.59, Q_cos=1.46
#  90: nll: 0.68/0.48, mi: -1.203/ 1.106, acc: 77.5/77.5, eps: 0.10569, R=[-1.44, -0.08, -1.06], t=0.48, Q_cos=1.79
Saved checkpoint
# 120: nll: 0.83/0.35, mi: -0.224/ 1.852, acc: 55.0/86.5, eps: 0.07759, R=[-2.0, -0.08, -0.32], t=0.40, Q_cos=2.02
Saved checkpoint
# 150: nll: 0.42/0.40, mi:  2.104/ 1.586, acc: 80.0/82.5, eps: 0.05346, R=[-2.0, -0.0, 0.36], t=0.35, Q_cos=2.03
# 180: nll: 0.39/0.35, mi:  1.298/ 1.954, acc: 87.5/86.7, eps: 0.00078, R=[-1.8, 0.06, 0.48], t=0.35, Q_cos=1.86
Saved checkpoint
#

(array([0.13677994, 0.02670323]), array([0.02442333, 0.00491133]))

In [32]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, True), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:3], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.75/0.65, mi:  0.045/ 0.645, acc: 52.5/66.7, eps: 0.56742, R=[-0.2, -0.26, -0.62], t=0.57, Q_cos=0.70
#  30: nll: 0.64/0.45, mi:  0.497/ 1.212, acc: 65.0/80.0, eps: 0.15769, R=[-1.24, 0.02, -1.41], t=0.41, Q_cos=1.88
Saved checkpoint
#  60: nll: 0.59/0.39, mi:  0.684/ 1.880, acc: 65.0/81.2, eps: 0.08704, R=[-1.57, 0.07, -0.5], t=0.32, Q_cos=1.65
Saved checkpoint
#  90: nll: 0.66/0.43, mi:  0.284/ 1.359, acc: 62.5/81.8, eps: 0.00283, R=[-1.33, 0.05, 0.05], t=0.32, Q_cos=1.33
# 120: nll: 0.40/0.40, mi:  1.603/ 1.471, acc: 80.0/84.2, eps: 0.04014, R=[-1.47, 0.24, 0.29], t=0.28, Q_cos=1.52
# 150: nll: 0.47/0.40, mi:  1.182/ 1.716, acc: 75.0/81.2, eps: 0.01273, R=[-1.51, 0.18, 0.24], t=0.26, Q_cos=1.54
# 180: nll: 0.61/0.32, mi: -0.042/ 2.201, acc: 62.5/87.5, eps: 0.04606, R=[-1.54, 0.23, 0.69], t=0.22, Q_cos=1.70
Saved checkpoint
# 210: nll: 0.39/0.3

(array([0.16348634, 0.0402617 ]), array([0.04123065, 0.00629843]))

In [30]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, True), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:4], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 1.19/0.64, mi: -1.062/ 0.226, acc: 40.0/63.0, eps: 0.56754, R=[-0.32, -0.26, -0.74], t=0.57, Q_cos=0.84
#  30: nll: 0.40/0.45, mi:  1.250/ 1.400, acc: 87.5/79.5, eps: 0.12256, R=[-1.19, -0.06, -1.06], t=0.44, Q_cos=1.59
Saved checkpoint
#  60: nll: 0.54/0.50, mi:  0.880/ 1.074, acc: 75.0/78.2, eps: 0.06625, R=[-1.63, 0.16, -0.42], t=0.38, Q_cos=1.69
#  90: nll: 0.81/0.41, mi: -0.254/ 1.458, acc: 47.5/81.2, eps: 0.01014, R=[-1.56, 0.26, 0.2], t=0.37, Q_cos=1.59
Saved checkpoint
# 120: nll: 0.57/0.36, mi:  0.783/ 1.667, acc: 62.5/85.8, eps: 0.02760, R=[-1.82, 0.23, 0.36], t=0.34, Q_cos=1.87
Saved checkpoint
# 150: nll: 0.68/0.40, mi:  0.073/ 1.632, acc: 70.0/83.0, eps: 0.01739, R=[-1.66, 0.27, 0.63], t=0.32, Q_cos=1.80
# 180: nll: 0.47/0.35, mi:  0.145/ 1.860, acc: 77.5/86.0, eps: 0.02028, R=[-1.59, 0.27, 0.5], t=0.30, Q_cos=1.69
Saved checkpoint
# 

(array([0.2343913 , 0.03457385]), array([0.0978024 , 0.00899344]))

# Testing number of frames (sampling / no grad)

In [10]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, False), (1000, 1e-4, False, False), (1000, 1e-5, False, False)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.965,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.57/0.68, mi:  0.494/ 0.199, acc: 67.5/61.5, eps: 0.58518, R=[-0.2, -0.26, -0.74], t=0.58, Q_cos=0.81
#  30: nll: 0.68/0.58, mi: -0.131/ 0.516, acc: 60.0/71.2, eps: 0.01986, R=[-0.36, -0.22, -0.76], t=0.60, Q_cos=0.87
Saved checkpoint
#  60: nll: 0.51/0.56, mi:  0.644/ 0.604, acc: 75.0/73.0, eps: 0.02603, R=[-0.52, -0.14, -0.77], t=0.63, Q_cos=0.94
Saved checkpoint
#  90: nll: 0.61/0.51, mi:  0.527/ 0.825, acc: 60.0/73.7, eps: 0.00534, R=[-0.56, -0.22, -1.05], t=0.64, Q_cos=1.21
Saved checkpoint
# 120: nll: 0.34/0.54, mi:  2.193/ 0.776, acc: 77.5/74.5, eps: 0.00864, R=[-0.75, -0.12, -1.4], t=0.63, Q_cos=1.59
# 150: nll: 0.53/0.52, mi:  0.658/ 0.744, acc: 75.0/73.8, eps: 0.04581, R=[-0.62, -0.15, -1.79], t=0.67, Q_cos=1.90
# 180: nll: 0.42/0.48, mi:  1.563/ 0.873, acc: 82.5/77.7, eps: 0.01055, R=[-0.7, -0.06, -2.35], t=0.68, Q_cos=2.46
Saved check

(array([2.88444716, 0.72867707]), array([1.47743205, 0.26726513]))

In [36]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, False), (1000, 1e-4, False, False), (1000, 1e-5, False, False)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.8,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.54/0.59, mi:  0.812/ 0.796, acc: 72.5/67.0, eps: 0.56535, R=[-0.32, -0.26, -0.62], t=0.57, Q_cos=0.75
#  30: nll: 0.43/0.40, mi:  2.249/ 1.554, acc: 70.0/84.8, eps: 0.15381, R=[-1.22, -0.11, -0.71], t=0.41, Q_cos=1.42
Saved checkpoint
#  60: nll: 0.33/0.35, mi:  2.088/ 2.052, acc: 92.5/85.3, eps: 0.14574, R=[-1.99, -0.1, -0.51], t=0.26, Q_cos=2.06
Saved checkpoint
#  90: nll: 0.45/0.46, mi:  1.174/ 1.356, acc: 77.5/79.3, eps: 0.01806, R=[-1.97, -0.05, -0.44], t=0.24, Q_cos=2.02
# 120: nll: 0.31/0.37, mi:  2.150/ 1.674, acc: 90.0/86.0, eps: 0.05670, R=[-1.46, -0.01, 0.09], t=0.30, Q_cos=1.47
# 150: nll: 0.36/0.43, mi:  1.352/ 1.431, acc: 82.5/80.0, eps: 0.03373, R=[-1.56, 0.01, 0.15], t=0.27, Q_cos=1.57
# 180: nll: 0.44/0.40, mi:  1.144/ 1.763, acc: 82.5/83.2, eps: 0.03810, R=[-1.78, 0.06, 0.19], t=0.23, Q_cos=1.79
# 210: nll: 0.73/0.35, mi:  0.1

(array([1.24929896, 0.13886765]), array([0.15227856, 0.03156138]))

In [35]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, False), (1000, 1e-4, False, False), (1000, 1e-5, False, False)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.5,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.81/0.65, mi: -0.432/ 0.431, acc: 50.0/63.2, eps: 0.58518, R=[-0.2, -0.26, -0.74], t=0.58, Q_cos=0.81
#  30: nll: 0.35/0.55, mi:  2.139/ 0.656, acc: 82.5/72.0, eps: 0.06740, R=[-0.82, -0.1, -0.55], t=0.52, Q_cos=0.99
Saved checkpoint
#  60: nll: 0.33/0.46, mi:  1.464/ 1.523, acc: 90.0/81.5, eps: 0.09635, R=[-1.35, -0.01, -0.35], t=0.42, Q_cos=1.39
Saved checkpoint
#  90: nll: 0.40/0.40, mi:  1.841/ 1.564, acc: 80.0/84.0, eps: 0.09606, R=[-1.79, 0.04, -0.36], t=0.32, Q_cos=1.82
Saved checkpoint
# 120: nll: 0.55/0.36, mi: -0.331/ 1.810, acc: 85.0/85.2, eps: 0.02820, R=[-1.97, -0.01, -0.35], t=0.29, Q_cos=2.00
Saved checkpoint
# 150: nll: 0.40/0.38, mi:  1.724/ 1.612, acc: 80.0/85.2, eps: 0.00629, R=[-1.94, 0.05, 0.13], t=0.29, Q_cos=1.94
# 180: nll: 0.38/0.42, mi:  1.753/ 1.414, acc: 82.5/81.0, eps: 0.02201, R=[-2.05, 0.04, 0.12], t=0.26, Q_cos=2.0

(array([2.50013183, 0.3860342 ]), array([2.01626725, 0.39491969]))

In [34]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, False), (1000, 1e-4, False, False), (1000, 1e-5, False, False)]:
    
        #ac.load_state_dict(torch.load("checkpoints/checkpoint_best_mi.pth.tar"))
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:1], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.81/0.65, mi: -0.432/ 0.431, acc: 50.0/63.2, eps: 0.58518, R=[-0.2, -0.26, -0.74], t=0.58, Q_cos=0.81
#  30: nll: 0.36/0.57, mi:  1.524/ 0.701, acc: 85.0/73.3, eps: 0.03817, R=[-0.38, -0.18, -0.49], t=0.55, Q_cos=0.64
Saved checkpoint
#  60: nll: 0.37/0.53, mi:  1.843/ 0.989, acc: 80.0/79.3, eps: 0.13331, R=[-1.36, -0.09, -0.81], t=0.41, Q_cos=1.59
Saved checkpoint
#  90: nll: 0.27/0.44, mi:  2.282/ 1.342, acc: 92.5/80.5, eps: 0.10632, R=[-2.01, -0.06, -0.77], t=0.30, Q_cos=2.15
Saved checkpoint
# 120: nll: 0.34/0.35, mi:  2.259/ 1.956, acc: 80.0/87.5, eps: 0.00935, R=[-1.7, 0.02, -0.68], t=0.31, Q_cos=1.83
Saved checkpoint
# 150: nll: 0.30/0.43, mi:  2.208/ 1.307, acc: 87.5/81.2, eps: 0.04754, R=[-1.71, 0.01, -0.68], t=0.27, Q_cos=1.84
# 180: nll: 0.38/0.37, mi:  1.762/ 1.824, acc: 82.5/83.2, eps: 0.02313, R=[-1.74, 0.01, -0.7], t=0.24, Q_cos=1.

(array([0.23745036, 0.1217819 ]), array([0.01157156, 0.00939478]))

In [33]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, False), (1000, 1e-4, False, False), (1000, 1e-5, False, False)]:
    
        #ac.load_state_dict(torch.load("checkpoints/checkpoint_best_mi.pth.tar"))
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:2], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.60/0.69, mi:  0.293/ 0.367, acc: 62.5/63.5, eps: 0.56818, R=[-0.32, -0.38, -0.74], t=0.57, Q_cos=0.89
#  30: nll: 0.78/0.61, mi: -0.097/ 0.498, acc: 52.5/69.3, eps: 0.03954, R=[-0.33, 0.07, -1.34], t=0.61, Q_cos=1.38
Saved checkpoint
#  60: nll: 0.76/0.59, mi:  0.023/ 0.294, acc: 60.0/72.2, eps: 0.01720, R=[-0.65, 0.06, -1.3], t=0.59, Q_cos=1.46
#  90: nll: 0.68/0.48, mi: -1.203/ 1.106, acc: 77.5/77.5, eps: 0.10569, R=[-1.44, -0.08, -1.06], t=0.48, Q_cos=1.79
Saved checkpoint
# 120: nll: 0.83/0.35, mi: -0.224/ 1.852, acc: 55.0/86.5, eps: 0.07759, R=[-2.0, -0.08, -0.32], t=0.40, Q_cos=2.02
Saved checkpoint
# 150: nll: 0.42/0.40, mi:  2.104/ 1.586, acc: 80.0/82.5, eps: 0.05346, R=[-2.0, -0.0, 0.36], t=0.35, Q_cos=2.03
# 180: nll: 0.39/0.35, mi:  1.298/ 1.954, acc: 87.5/86.7, eps: 0.00078, R=[-1.8, 0.06, 0.48], t=0.35, Q_cos=1.86
Saved checkpoint
#

(array([0.13677994, 0.02670323]), array([0.02442333, 0.00491133]))

In [32]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, False), (1000, 1e-4, False, False), (1000, 1e-5, False, False)]:
    
        #ac.load_state_dict(torch.load("checkpoints/checkpoint_best_mi.pth.tar"))
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:3], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 0.75/0.65, mi:  0.045/ 0.645, acc: 52.5/66.7, eps: 0.56742, R=[-0.2, -0.26, -0.62], t=0.57, Q_cos=0.70
#  30: nll: 0.64/0.45, mi:  0.497/ 1.212, acc: 65.0/80.0, eps: 0.15769, R=[-1.24, 0.02, -1.41], t=0.41, Q_cos=1.88
Saved checkpoint
#  60: nll: 0.59/0.39, mi:  0.684/ 1.880, acc: 65.0/81.2, eps: 0.08704, R=[-1.57, 0.07, -0.5], t=0.32, Q_cos=1.65
Saved checkpoint
#  90: nll: 0.66/0.43, mi:  0.284/ 1.359, acc: 62.5/81.8, eps: 0.00283, R=[-1.33, 0.05, 0.05], t=0.32, Q_cos=1.33
# 120: nll: 0.40/0.40, mi:  1.603/ 1.471, acc: 80.0/84.2, eps: 0.04014, R=[-1.47, 0.24, 0.29], t=0.28, Q_cos=1.52
# 150: nll: 0.47/0.40, mi:  1.182/ 1.716, acc: 75.0/81.2, eps: 0.01273, R=[-1.51, 0.18, 0.24], t=0.26, Q_cos=1.54
# 180: nll: 0.61/0.32, mi: -0.042/ 2.201, acc: 62.5/87.5, eps: 0.04606, R=[-1.54, 0.23, 0.69], t=0.22, Q_cos=1.70
Saved checkpoint
# 210: nll: 0.39/0.3

(array([0.16348634, 0.0402617 ]), array([0.04123065, 0.00629843]))

In [30]:
np.random.seed(2)
log = []

for i in range(5):
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         pc_batch=5000, mi_batch=20, 
                         patch_size=96, hidden_size=32,  
                         true_kittis=None, max_z_true_kittis=30
                         )
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.reset_calibration()

    interval_t = np.random.uniform(-0.6, 0.6, (3,))
    interval_r = np.random.uniform(-2, 2, (3,))
    print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
    ac.distort_calibration(interval_t, interval_r)
    
    for iters, lr, grad, sample in [(2000, 1e-3, False, False), (1000, 1e-4, False, False), (1000, 1e-5, False, False)]:
    
        ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
        ac.best_mi = 0.
        r = ac.fit(datas_test1[0:4], iters=iters, log_interval=30, viz="none", 
                   log_avg_window=1,
                   n_test_batches=10, 
                   valid_ratio=0.2,
                   grad_mine=grad,
                   grad_SE=True, 
                   grad_R=True,
                   grad_T=True,
                   weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000), 
                   sample_weight_pool=sample)

    log.append((r, interval_t, interval_r))
    
metrics = [] # 0.50 / 5
i  = 1
for r, interval_t, interval_r in log:
    
    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]
    
    metrics.append([Q_cos, t_l2])
    
    print(interval_r, interval_t)
    
np.asarray(metrics).mean(0), np.asarray(metrics).std(0)


 Distortion t:  [-0.07680612 -0.56888852  0.05959497] r:  [-0.25871043 -0.31852879 -0.67866072] 

MINE weight pool size is 2000
#   0: nll: 1.19/0.64, mi: -1.062/ 0.226, acc: 40.0/63.0, eps: 0.56754, R=[-0.32, -0.26, -0.74], t=0.57, Q_cos=0.84
#  30: nll: 0.40/0.45, mi:  1.250/ 1.400, acc: 87.5/79.5, eps: 0.12256, R=[-1.19, -0.06, -1.06], t=0.44, Q_cos=1.59
Saved checkpoint
#  60: nll: 0.54/0.50, mi:  0.880/ 1.074, acc: 75.0/78.2, eps: 0.06625, R=[-1.63, 0.16, -0.42], t=0.38, Q_cos=1.69
#  90: nll: 0.81/0.41, mi: -0.254/ 1.458, acc: 47.5/81.2, eps: 0.01014, R=[-1.56, 0.26, 0.2], t=0.37, Q_cos=1.59
Saved checkpoint
# 120: nll: 0.57/0.36, mi:  0.783/ 1.667, acc: 62.5/85.8, eps: 0.02760, R=[-1.82, 0.23, 0.36], t=0.34, Q_cos=1.87
Saved checkpoint
# 150: nll: 0.68/0.40, mi:  0.073/ 1.632, acc: 70.0/83.0, eps: 0.01739, R=[-1.66, 0.27, 0.63], t=0.32, Q_cos=1.80
# 180: nll: 0.47/0.35, mi:  0.145/ 1.860, acc: 77.5/86.0, eps: 0.02028, R=[-1.59, 0.27, 0.5], t=0.30, Q_cos=1.69
Saved checkpoint
# 

(array([0.2343913 , 0.03457385]), array([0.0978024 , 0.00899344]))